In [29]:
# Name: Mohammad Naufal Eiman bin Shahromi , Siti Nur Lyana binti Mohd Nazri
# Student ID: IS01082514 , IS01082645

import pandas as pd

# Load dataset
df = pd.read_csv("Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [31]:
df = df[['Score', 'Text']]

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [33]:
def score_to_sentiment(score):
    if score <= 2:
        return "negative"
    elif score == 3:
        return "neutral"
    else:
        return "positive"

df['Sentiment'] = df['Score'].apply(score_to_sentiment)
df = df[['Text', 'Sentiment']]
df.head()

,Text,Sentiment
0,I have bought several of the Vitality canned d...,positive
1,Product arrived labeled as Jumbo Salted Peanut...,negative
2,This is a confection that has been around a fe...,positive
3,If you are looking for the secret ingredient i...,negative
4,Great taffy at a great price. There was a wid...,positive


In [35]:
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)  
    text = text.translate(str.maketrans('', '', string.punctuation))  
    text = ' '.join([word for word in text.split() if word not in stop_words])  
    return text

df['Cleaned_Text'] = df['Text'].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\naufa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Text,Sentiment,Cleaned_Text
0,I have bought several of the Vitality canned d...,positive,bought several vitality canned dog food produc...
1,Product arrived labeled as Jumbo Salted Peanut...,negative,product arrived labeled jumbo salted peanutsth...
2,This is a confection that has been around a fe...,positive,confection around centuries light pillowy citr...
3,If you are looking for the secret ingredient i...,negative,looking secret ingredient robitussin believe f...
4,Great taffy at a great price. There was a wid...,positive,great taffy great price wide assortment yummy ...


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['Cleaned_Text'])

y = df['Sentiment']

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [43]:
from sklearn.naive_bayes import MultinomialNB
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

MultinomialNB()

In [45]:
from sklearn.metrics import classification_report, confusion_matrix

# Logistic Regression
print("Logistic Regression Results:")
y_pred_lr = model_lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

# Naive Bayes
print("Naive Bayes Results:")
y_pred_nb = model_nb.predict(X_test)
print(classification_report(y_test, y_pred_nb))

Logistic Regression Results:
              precision    recall  f1-score   support

    negative       0.73      0.67      0.70     16181
     neutral       0.51      0.19      0.28      8485
    positive       0.90      0.97      0.93     89025

    accuracy                           0.87    113691
   macro avg       0.71      0.61      0.64    113691
weighted avg       0.85      0.87      0.85    113691

Naive Bayes Results:
              precision    recall  f1-score   support

    negative       0.83      0.27      0.40     16181
     neutral       0.40      0.00      0.01      8485
    positive       0.82      1.00      0.90     89025

    accuracy                           0.82    113691
   macro avg       0.68      0.42      0.44    113691
weighted avg       0.79      0.82      0.76    113691



In [47]:
df_cleaned = df[['Cleaned_Text', 'Sentiment']]
df_cleaned.to_csv("cleaned_amazon_reviews.csv", index=False)

Name: Mohammad Naufal Eiman bin Shahromi , Siti Nur Lyana binti Mohd Nazri
Student ID: IS01082514 , IS01082645 

Step 5: Discussion

In this sentiment analysis assignment, both machine learning models (Logistic Regression and Naive Bayes) showed decent performance. Logistic Regression achieved higher accuracy and better precision/recall for the positive and negative classes, likely because it handles sparse TF-IDF features more effectively.

Naive Bayes, while faster and simpler, struggled slightly with the neutral class due to its strong independence assumptions. It performed reasonably well on the dominant "positive" class, which was expected due to class imbalance.

Lexicon-based approaches like TextBlob or VADER (not shown in this notebook) are useful for quick insights or smaller datasets, but they often fail to capture context, sarcasm, or domain-specific language — especially in long Amazon reviews.

Overall, machine learning models—especially Logistic Regression—were more robust for this dataset and better at generalizing to new, unseen reviews.
